<a href="https://colab.research.google.com/github/psgrigoletti/margem-liquida-colab/blob/main/plotly_teste_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kora

     |████████████████████████████████| 57 kB 2.0 MB/s 
     |████████████████████████████████| 56 kB 2.7 MB/s 


In [2]:
!pip install -U plotly


     |████████████████████████████████| 27.7 MB 1.4 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [25]:
from kora.selenium import wd
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from google.colab.data_table import DataTable
from time import sleep
import plotly.express as px

In [59]:
def busca_carteira_teorica(indice, espera=8):
  url = f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{indice.upper()}/?language=pt-br'
  wd.get(url)
  sleep(espera)
  wd.find_element(by=By.ID, value="segment").send_keys('Setor de Atuação')
  sleep(espera)
  wd.find_element(by=By.LINK_TEXT, value="Download").click()
  sleep(espera)
  arquivos = !ls -1t *.csv
  return pd.read_csv(arquivos[0], sep=";", encoding='ISO-8859-1', skipfooter=2, engine="python", thousands=".", decimal=",", header=1, index_col=False)

In [60]:
ibov = busca_carteira_teorica("ibov", 3)
ibov

,Setor,Código,Ação,Tipo,Qtde. Teórica,Part. (%),Part. (%)Acum.
0,Bens Indls / Máqs e Equips,WEGE3,WEG,ON ED NM,1484859030,2.048,2.048
1,Bens Indls / Mat Transporte,EMBR3,EMBRAER,ON NM,734558205,0.656,0.656
2,Bens Indls/Transporte,AZUL4,AZUL,PN N2,327741172,0.439,2.228
3,Bens Indls/Transporte,CCRO3,CCR SA,ON NM,1115693556,0.670,2.228
4,Bens Indls/Transporte,ECOR3,ECORODOVIAS,ON NM,339237914,0.110,2.228
...,...,...,...,...,...,...,...
86,Utilidade Públ / Energ Elétrica,ENGI11,ENERGISA,UNT N2,248477689,0.507,6.009
87,Utilidade Públ / Energ Elétrica,ENEV3,ENEVA,ON NM,1261568398,0.803,6.009
88,Utilidade Públ / Energ Elétrica,EGIE3,ENGIE BRASIL,ON NM,255217329,0.481,6.009
89,Utilidade Públ / Energ Elétrica,EQTL3,EQUATORIAL,ON NM,1010511085,1.175,6.009


In [61]:
ifix = busca_carteira_teorica("ifix", 3)
ifix

,Setor,Código,Ação,Tipo,Qtde. Teórica,Part. (%),Part. (%)Acum.
0,Financ e Outros / Fundos,BPFF11,FII ABSOLUTO,CI,4492326,0.277,100.0
1,Financ e Outros / Fundos,AFHI11,FII AFHI CRI,CI ER,1787671,0.171,100.0
2,Financ e Outros / Fundos,ALZR11,FII ALIANZA,CI ER,5922011,0.655,100.0
3,Financ e Outros / Fundos,ARCT11,FII ARCTIUM,CI ER,3597568,0.371,100.0
4,Financ e Outros / Fundos,AIEC11,FII AUTONOMY,CI,4824987,0.355,100.0
...,...,...,...,...,...,...,...
99,Financ e Outros / Fundos,XPLG11,FII XP LOG,CI,27101061,2.508,100.0
100,Financ e Outros / Fundos,XPCM11,FII XP MACAE,CI,2414570,0.041,100.0
101,Financ e Outros / Fundos,XPML11,FII XP MALLS,CI ER,18681577,1.659,100.0
102,Financ e Outros / Fundos,XPPR11,FII XP PROP,CI,7316171,0.414,100.0


In [62]:
ibov['SubSetor'] = ibov['Setor'].apply(lambda s: s[s.rfind("/")+1:].strip())
ibov['Setor'] = ibov['Setor'].apply(lambda s: s[:s.rfind("/")].strip())
ibov['Setor'].unique()

ifix['SubSetor'] = ifix['Setor'].apply(lambda s: s[s.rfind("/")+1:].strip())
ifix['Setor'] = ifix['Setor'].apply(lambda s: s[:s.rfind("/")].strip())
ifix['Setor'].unique()

array(['Financ e Outros'], dtype=object)

In [64]:
ibov['SubSetor'].unique()
ifix['SubSetor'].unique()

array(['Fundos'], dtype=object)

In [69]:
fig = px.sunburst(data_frame=ibov, path=['Setor', 'SubSetor', 'Código'], values="Part. (%)", height=900)
fig.update_traces(textfont_color="white", textfont_size=14, 
                  hovertemplate="<b>%{label}:</b> %{value:.2f}%")
fig.show()
f = open("grafico.html", "w")
f.write(fig.to_html())
f.close()

In [68]:
fig = px.treemap(data_frame=ifix, path=['Setor', 'SubSetor', 'Código'], values="Part. (%)", height=900)
fig.update_traces(textfont_color="white", textfont_size=14, 
                  hovertemplate="<b>%{label}:</b> %{value:.2f}%")
fig.show()
#fig.to_html()